In [ ]:
#Parser for directory
import os
import xml.etree.ElementTree as ET
import logging

class Task:
    def __init__(self, id, name, incoming, outgoing):
        self.id = id
        self.name = name
        self.incoming = incoming
        self.outgoing = outgoing

class Gateway:
    def __init__(self, id, name, incoming, outgoing):
        self.id = id
        self.name = name
        self.incoming = incoming
        self.outgoing = outgoing

class StartEvent:
    def __init__(self, id, name, incoming, outgoing):
        self.id = id
        self.name = name
        self.incoming = incoming
        self.outgoing = outgoing

class EndEvent:
    def __init__(self, id, name, incoming, outgoing):
        self.id = id
        self.name = name
        self.incoming = incoming
        self.outgoing = outgoing

def parse_bpmn(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    tasks = []
    gateways = []
    start_events = []
    end_events = []

    ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

    for element in root.findall('.//bpmn:task', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            name = 'Task' + f'{len(tasks)}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        task = Task(id, name, incoming, outgoing)
        tasks.append(task)

    for element in root.findall('.//bpmn:exclusiveGateway', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            name = 'ExclusiveGateway' + f'{len(gateways)}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)
        
    for element in root.findall('.//bpmn:eventBasedGateway', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            name = 'EventBasedGateway' + f'{len(gateways)}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)
        
    for element in root.findall('.//bpmn:inclusiveGateway', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            name = 'InclusiveGateway' + f'{len(gateways)}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)
        
    for element in root.findall('.//bpmn:parallelGateway', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            name = 'ParallelGateway' + f'{len(gateways)}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)
        
    for element in root.findall('.//bpmn:intermediateCatchEvent', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            name = 'IntermediateCatchEvent' + f'{len(gateways)}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)

    for element in root.findall('.//bpmn:startEvent', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            name = 'StartEvent' + f'{len(start_events)}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        start_event = StartEvent(id, name, incoming, outgoing)
        start_events.append(start_event)

    for element in root.findall('.//bpmn:endEvent', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            name = 'EndEvent' + f'{len(end_events)}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        end_event = EndEvent(id, name, incoming, outgoing)
        end_events.append(end_event)

    return tasks, gateways, start_events, end_events

def process_bpmn_objects(tasks, gateways, start_events, end_events):
    for task in tasks:
        for x in range(len(tasks)):
            len_task_in = len(task.incoming)
            len_task_out = len(task.outgoing)
            for length_in in range(len_task_in):
                for length_out in range(len_task_out):
                    if task.outgoing[length_out] == tasks[x].incoming[length_in]:
                        task.outgoing[length_out] = tasks[x].name
                        tasks[x].incoming[length_in] = task.name
        for gateway in gateways:
            length_out_gate = len(gateway.outgoing)
            len_task_in = len(task.incoming)
            for length_in in range(len_task_in):
                for length_gate in range(length_out_gate):
                    if task.incoming[length_in] == gateway.outgoing[length_gate]:
                        gateway.outgoing[length_gate] = task.name
                        task.incoming[length_in] = gateway.name
            length_in_gate = len(gateway.incoming)
            len_task_out = len(task.outgoing)
            for length_gate in range(length_in_gate):
                for length_out in range(len_task_out):
                    if task.outgoing[length_out] == gateway.incoming[length_gate]:
                        task.outgoing[length_out] = gateway.name
                        gateway.incoming[length_gate] = task.name
            for gateway2 in gateways:
                length_in_gate = len(gateway2.incoming)
                length_out_gate = len(gateway.outgoing)
                for length_gate_in in range(length_in_gate):
                    for length_gate_out in range(length_out_gate):
                        if gateway2.incoming[length_gate_in] == gateway.outgoing[length_gate_out]:
                            gateway2.incoming[length_gate_in] = gateway.name
                            gateway.outgoing[length_gate_out] = gateway2.name
        for start_event in start_events:
            len_task_in = len(task.incoming)
            start_len_out = len(start_event.outgoing)
            for start_event_out in range(start_len_out):
                for length_in in range(len_task_in):
                    if task.incoming[length_in] == start_event.outgoing[start_event_out]:
                        task.incoming[length_in] = start_event.name
                        start_event.outgoing[start_event_out] = task.name
                for gateway in gateways:
                    for gateway_in in range(len(gateway.incoming)):
                        if gateway.incoming[gateway_in] == start_event.outgoing[start_event_out]:
                            gateway.incoming[gateway_in] = start_event.name
                            start_event.outgoing[start_event_out] = gateway.name
        for end_event in end_events:
            len_task_out = len(task.outgoing)
            end_len_in = len(end_event.incoming)
            for end_event_in in range(end_len_in):
                for length_out in range(len_task_out):
                    if task.outgoing[length_out] == end_event.incoming[end_event_in]:
                        task.outgoing[length_out] = end_event.name
                        end_event.incoming[end_event_in] = task.name
                for gateway in gateways:
                    for gateway_out in range(len(gateway.outgoing)):
                        if gateway.outgoing[gateway_out] == end_event.incoming[end_event_in]:
                            gateway.outgoing[gateway_out] = end_event.name
                            end_event.incoming[end_event_in] = gateway.name

def save_processed_objects(file_path, tasks, gateways, start_events, end_events):
    with open(file_path, 'w') as f:
        f.write("Tasks:\n")
        for task in tasks:
            f.write(str(vars(task)) + "\n")
        f.write("\nGateways:\n")
        for gateway in gateways:
            f.write(str(vars(gateway)) + "\n")
        f.write("\nStart Events:\n")
        for start_event in start_events:
            f.write(str(vars(start_event)) + "\n")
        f.write("\nEnd Events:\n")
        for end_event in end_events:
            f.write(str(vars(end_event)) + "\n")

def process_all_bpmn_files(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(input_directory):
        if filename.endswith(".bpmn"):
            input_file_path = os.path.join(input_directory, filename)
            output_file_path = os.path.join(output_directory, f"processed_{filename}.txt")

            try:
                tasks, gateways, start_events, end_events = parse_bpmn(input_file_path)
                process_bpmn_objects(tasks, gateways, start_events, end_events)
                save_processed_objects(output_file_path, tasks, gateways, start_events, end_events)
                print(f"Processed {filename} and saved to {output_file_path}")
            except Exception as e:
                logging.error(f"Error processing {filename}: {e}")

# Set up logging
logging.basicConfig(filename='bpmn_processing.log', level=logging.ERROR)

# Directory containing .bpmn files
input_directory = './the process/'
# Directory to save processed files
output_directory = './output_newbranches/'

# Process all .bpmn files in the input directory
process_all_bpmn_files(input_directory, output_directory)


In [ ]:
#Parser for individual files
import xml.etree.ElementTree as ET

class Task:
    def __init__(self, id, name, incoming, outgoing):
        self.id = id
        self.name = name
        self.incoming = incoming
        self.outgoing = outgoing

class Gateway:
    def __init__(self, id, name, incoming, outgoing):
        self.id = id
        self.name = name
        self.incoming = incoming
        self.outgoing = outgoing

class StartEvent:
    def __init__(self, id, name, incoming, outgoing):
        self.id = id
        self.name = name
        self.incoming = incoming
        self.outgoing = outgoing

class EndEvent:
    def __init__(self, id, name, incoming, outgoing):
        self.id = id
        self.name = name
        self.incoming = incoming
        self.outgoing = outgoing

def parse_bpmn(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    tasks = []
    gateways = []
    start_events = []
    end_events = []

    # Namespace dictionary for finding elements with namespaces
    ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

    # Parse tasks
    for element in root.findall('.//bpmn:task', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            created_name = 'Task'+f'{len(gateways)}'
            name = f'{created_name}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        task = Task(id, name, incoming, outgoing)
        tasks.append(task)

    # Parse gateways
    for element in root.findall('.//bpmn:exclusiveGateway', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            created_name = 'ExclusiveGateway'+f'{len(gateways)}'
            name = f'{created_name}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)
        
    for element in root.findall('.//bpmn:eventBasedGateway', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            created_name = 'EventBasedGateway'+f'{len(gateways)}'
            name = f'{created_name}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)
        
    for element in root.findall('.//bpmn:inclusiveGateway', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            created_name = 'InclusivedGateway'+f'{len(gateways)}'
            name = f'{created_name}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)
        
    for element in root.findall('.//bpmn:parallelGateway', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            created_name = 'ParallelGateway'+f'{len(gateways)}'
            name = f'{created_name}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)
        
    for element in root.findall('.//bpmn:intermediateCatchEvent', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            created_name = 'IntermediateCatchEvent'+f'{len(gateways)}'
            name = f'{created_name}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        gateway = Gateway(id, name, incoming, outgoing)
        gateways.append(gateway)

    # Parse start events
    for element in root.findall('.//bpmn:startEvent', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            created_name = 'StartEvent'+f'{len(gateways)}'
            name = f'{created_name}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        start_event = StartEvent(id, name, incoming, outgoing)
        start_events.append(start_event)

    # Parse end events
    for element in root.findall('.//bpmn:endEvent', ns):
        id = element.attrib.get('id')
        name = element.attrib.get('name', '')
        if name == '':
            created_name = 'EndEvent'+f'{len(gateways)}'
            name = f'{created_name}'
        incoming = [e.text for e in element.findall('bpmn:incoming', ns)]
        outgoing = [e.text for e in element.findall('bpmn:outgoing', ns)]
        end_event = EndEvent(id, name, incoming, outgoing)
        end_events.append(end_event)

    return tasks, gateways, start_events, end_events

    


# Example usage
file_path = 'uebung2_a5605713f817474d99a198ad61d52387_translated.bpmn'
tasks, gateways, start_events, end_events = parse_bpmn(file_path)

for task in tasks:
        for x in range(len(tasks)):
            len_task_in = 0
            len_task_out = 0
            len_task_in = len(task.incoming)
            len_task_out = len(task.outgoing)
            for length_in in range(len_task_in):
                for length_out in range(len_task_out):
                    if task.outgoing[length_out] == tasks[x].incoming[length_in]:
                        task.outgoing[length_out]=tasks[x].name
                        tasks[x].incoming[length_in]=task.name
        for gateway in gateways:
            length_out_gate = 0
            length_out_gate = len(gateway.outgoing)
            len_task_in = 0
            len_task_in = len(task.incoming)
            for length_in in range(len_task_in):
                for length_gate in range(length_out_gate):
                    if task.incoming[length_in] == gateway.outgoing[length_gate]:
                        gateway.outgoing[length_gate]=task.name
                        task.incoming[length_in]=gateway.name
            length_in_gate = 0
            length_in_gate = len(gateway.incoming)
            len_task_out = 0
            len_task_out = len(task.outgoing)
            for length_gate in range(length_in_gate):
                for length_out in range(len_task_out):
                    if task.outgoing[length_out] == gateway.incoming[length_gate]:
                        task.outgoing[length_out] = gateway.name
                        gateway.incoming[length_gate] = task.name
            for gateway2 in gateways:
                length_in_gate = 0
                length_in_gate = len(gateway2.incoming)
                length_out_gate = 0
                length_out_gate = len(gateway.outgoing)
                for length_gate_in in range(length_in_gate):
                    for length_gate_out in range(length_out_gate):
                        if gateway2.incoming[length_gate_in] == gateway.outgoing[length_gate_out]:
                            gateway2.incoming[length_gate_in] = gateway.name
                            gateway.outgoing[length_gate_out] = gateway2.name
        for start_event in start_events:
            len_task_in = len(task.incoming)
            start_len_out = len(start_event.outgoing)
            for start_event_out in range(start_len_out):
                for length_in in range(len_task_in):
                    if task.incoming[length_in] == start_event.outgoing[start_event_out]:
                        task.incoming[length_in] = start_event.name
                        start_event.outgoing[start_event_out] = task.name
                for gateway in gateways:
                    for gateway_in in range(len(gateway.incoming)):
                        if gateway.incoming[gateway_in]== start_event.outgoing[start_event_out]:
                            gateway.incoming[gateway_in] = start_event.name
                            start_event.outgoing[start_event_out]= gateway.name
        for end_event in end_events:
            len_task_out = len(task.outgoing)
            end_len_in = len(end_event.incoming)
            for end_event_in in range(end_len_in):
                for length_out in range(len_task_out):
                    if task.outgoing[length_out] == end_event.incoming[end_event_in]:
                        task.outgoing[length_out] = end_event.name
                        end_event.incoming[end_event_in] = task.name
                for gateway in gateways:
                    for gateway_out in range(len(gateway.outgoing)):
                        if gateway.outgoing[gateway_out]== end_event.incoming[end_event_in]:
                            gateway.outgoing[gateway_out] = end_event.name
                            end_event.incoming[end_event_in]= gateway.name

# Print parsed objects for verification
print("Tasks:")
for task in tasks:
    print(vars(task))

print("\nGateways:")
for gateway in gateways:
    print(vars(gateway))

print("\nStart Events:")
for start_event in start_events:
    print(vars(start_event))

print("\nEnd Events:")
for end_event in end_events:
    print(vars(end_event))
